In [2]:
from preprocessing import Preprocessor
from data_loader import load_json_data
from evaluation import compute_f1_scores, per_tag_f1
from baseline import tf_idf
import numpy as np

In [3]:
focus_tags=['math', 'graphs', 'strings', 'number theory', 'trees', 'geometry', 'games', 'probabilities']
data=load_json_data(r"C:\Users\maloc\OneDrive\Documents\Tag Classification\Code-Tag-Classification\data\code_classification_dataset.zip")

# TF IDF baseline model

First baseline mdoel is an invese frequency model, the data is whole descriptions or source codes and targets are the corresponding tags, minimal preprocessing is applied (cleaning text)

In [4]:
preprocessor=Preprocessor(use_code=False)
train_text, dev_text, test_text, train_labels, dev_labels, test_labels=preprocessor.process_data(data,devset=True,test_size=0.3) # type: ignore
model_tf=tf_idf()


=== CHECKING SPLIT BALANCE ===

=== Focus Tags Statistics ===
math            | train: 1012 dev: 194 test: 203
graphs          | train: 385 dev:  81 test:  76
strings         | train: 304 dev:  65 test:  53
number theory   | train: 250 dev:  53 test:  47
trees           | train: 212 dev:  60 test:  52
geometry        | train: 120 dev:  21 test:  25
games           | train:  66 dev:  21 test:  18
probabilities   | train:  65 dev:  16 test:  11
TRAIN SIZE: 3487
DEV SIZE: 747
TEST SIZE: 748


We see great imbalances in classes, need to tune threshold per classes for best results probably

In [5]:
model_tf.fit(train_text,train_labels)
mlb_classes=model_tf.mlb.classes_
y_true_binary=model_tf.mlb.transform(test_labels)

In [6]:
# Prediction using base threshold p=0.5

y_pred_binary=model_tf.predict(test_text)
per_tag_f1(y_true_binary,y_pred_binary,focus_tags=focus_tags,mlb_classes=mlb_classes)

[('math', 0.55),
 ('graphs', 0.47),
 ('strings', 0.59),
 ('number theory', 0.32),
 ('trees', 0.58),
 ('geometry', 0.08),
 ('games', 0.2),
 ('probabilities', 0.0)]

In [7]:
# Tune threshold globally to get better results, most classes are low count so p=0.5 is quite bad 

best_thresh,best_f1=model_tf.tune_threshold(dev_text,dev_labels,depth=10)

y_pred_binary=model_tf.predict(test_text)
per_tag_f1(y_true_binary,y_pred_binary,focus_tags=focus_tags,mlb_classes=mlb_classes)

[('math', 0.44),
 ('graphs', 0.42),
 ('strings', 0.63),
 ('number theory', 0.51),
 ('trees', 0.64),
 ('geometry', 0.56),
 ('games', 0.71),
 ('probabilities', 0.0)]

In [8]:
# Tune threshold per class to get best results

best_thresholds,best_f1_per_class=model_tf.tune_per_tag_threshold(dev_text,dev_labels,depth=20)
y_pred_binary=model_tf.predict(test_text)

per_tag_f1(y_true_binary,y_pred_binary,focus_tags=focus_tags,mlb_classes=mlb_classes)

[('math', 0.55),
 ('graphs', 0.54),
 ('strings', 0.63),
 ('number theory', 0.47),
 ('trees', 0.62),
 ('geometry', 0.58),
 ('games', 0.73),
 ('probabilities', 0.0)]

# Transformer model